In [ ]:

# Simulate hardware computation of dynamic dilated point method

import numpy as np
from numpy import random
import math
import pdb

random.seed ( 0)
# Predefined

pointcloud = np.array([[random.randint(-127, 128) for col in range(0, 3)] for row in range(0, 1024)]) # 8bit range
K = 24

SRAM_storage = 30*1024 # 30 KB
num_layer = 16
ALU = 1

# reserved inner layers for the points of current layer: determine when drop out inner layer
num_inner_layer_margin = 2 

print(pointcloud[0])

def distance_euclidean(coordinate0, coordinate1):
    coordinate0 = np.array(coordinate0)
    coordinate1 = np.array(coordinate1)

    return np.sqrt(np.sum(np.square(coordinate1-coordinate0)))
def sort_pointcloud_by_distance(center, pointcloud): # sort according to distance of diff coord. but coord. not changed
    '''
    input: <pointcloud> the 2D array of (3D_coordinate+C); e.g. [ [0,0,0, C], [] ]
    output: the sorted 2D array of (3D_coordinate+C+ sorted distance) array
    '''
    coord_diff = pointcloud[:, 0: 3] - center[0: 3]
    num_point = pointcloud.shape[0]
    pointcloud_distance = np.zeros([num_point])
    for idx in range(num_point):
        pointcloud_distance[idx] = distance_euclidean([0, 0, 0], coord_diff[idx])
    # pdb.set_trace()
    pointcloud_distance = pointcloud_distance.reshape(num_point, 1)

    # pointcloud[:, 0: 3] = coord_diff # replace with diff coord
    pointcloud = np.c_[pointcloud, pointcloud_distance]


    return pointcloud[np.lexsort(pointcloud.T)]

: 

In [ ]:
# Test lexsort
array = np.array([[1, 3, 2], [10, 9, 1]])
array[array.lexsort(array.T)]

In [ ]:

# Intermediate variable
num_point = pointcloud.shape[0]
center_point = [0, 0, 0]

# Sort according to distance to (0, 0, 0) 
pdb.set_trace()
pointcloud = sort_pointcloud_by_distance(np.array([0, 0, 0]), pointcloud)

pointcloud = np.c_[pointcloud, np.arange(num_point).reshape(num_point, 1)] # [3D coord., dist, idx]

print(pointcloud[0])
print(pointcloud[:, 3:4])
# sorted_idx = np.argsort(pointcloud_distance)
# for idx in range(num_point):
#     sorted_pointcloud[idx] = pointcloud[sorted_idx[idx]]

# coordinate sets of splitted layers

num_point_per_layer = math.ceil(num_point/num_layer)
layers = np.zeros([num_layer, num_point_per_layer, pointcloud.shape[1]])
for idx in range( num_layer ):
    layers[idx] = pointcloud[num_point_per_layer*idx : num_point_per_layer*(idx+1)]



In [ ]:

# Initalize

# On-chip
idx_layer = [0]
cur_layer = layers[idx_layer[0]] # [3D coord., dist, idx]


knn_map = []
knn_map_idx = []

# Statistic Results
stats_offchip_access = 0
stats_computation_mul = 0
stats_computation_add = 0
stats_computation_cycle = 0
for cur_idx in range(num_point): # serial processing all points according sorted distance

    stats_computation_add += 1
    ###########################################
    # position of cur_idx induced supplement inner/outer layer
    while ( (idx_layer[0] != 0) and cur_idx < cur_layer[0][4]): # # supplement inter layer
        idx_layer.insert(0, idx_layer[0]-1)
        cur_layer = np.r_[layers[idx_layer[0]], cur_layer]
        stats_offchip_access += np.size(layers[idx_layer[0]])
    while ( (idx_layer[-1] != num_layer -1) and cur_idx > cur_layer[-1][4]): 
        idx_layer.append(idx_layer[-1]+1)
        cur_layer = np.r_[cur_layer, layers[idx_layer[-1]]]
        stats_offchip_access += np.size(layers[idx_layer[-1]])

    ###########################################
    # position of cur_idx induced dropout inner layer
    for i_idx_layer in idx_layer:
        if cur_idx in cur_layer[i_idx_layer][:, 4].reshape(1, cur_layer.shape[0]):
            num_dropout_inner_layer = i_idx_layer-idx_layer[0]-num_inner_layer_margin
            if num_dropout_inner_layer > 0:
                del idx_layer[: num_dropout_inner_layer]
                cur_layer = np.delete(cur_layer, [0: num_point_per_layer*num_dropout_inner_layer], axis=0)
    

    # First step: knn sort in cur_layer
    # diff_cur_layer = cur_layer[0: 3] - pointcloud[cur_idx][0: 3]

    # for i in cur_layer.shape[0]: # which layer
    #     for j in cur_layer.shape[1]: # which point
    #         distance_cur_layer2cur_point[i*cur_layer.shape[1] + j] = distance_euclidean(pointcloud[cur_idx][0: 3], cur_layer[i][j][0: 3])
    # distance_cur_layer2cur_point = distance_cur_layer2cur_point.reshape(cur_layer.shape[0]*cur_layer.shape[1], 1)

    # cur_layer_temp = np.c_[cur_layer.reshape((cur_layer.shape[0]*cur_layer.shape[1], pointcloud.shape[1])), distance_cur_layer2cur_point]
    # cur_layer_temp[np.lexsort(cur_layer_temp.T)]
    pdb.set_trace()
    sorted_cur_layer_by_distance2cur_point = sort_pointcloud_by_distance(pointcloud[cur_idx], cur_layer) # [3D coord., dist, idx, distance]
    stats_offchip_access += np.size(pointcloud[cur_idx]) 

    # diff + square_add + sort
    stats_computation_add += np.size(cur_layer, 0)*3 + np.size(cur_layer, 0)*2 + np.size(cur_layer, 0)^2/2
    # square + sqrt = mul*4
    stats_computation_mul += np.size(cur_layer, 0)*3 + np.size(cur_layer, 0)*4

    stats_computation_cycle += np.size(cur_layer, 0)^2/2 / ALU
    # Second step: supplement sorted neighbor points by neighbor layers
    for k in range(K):
        # pdb.set_trace()
        # distance_cur2center = distance_euclidean((0, 0, 0), pointcloud[cur_idx][0: 3])
        # distance_cur2inter =   distance_cur2center - distance_euclidean((0, 0, 0), cur_layer[0][0][0: 3])
        # distance_cur2outer = -(distance_cur2center - distance_euclidean((0, 0, 0), cur_layer[-1][-1][0: 3]))
        while   ( (idx_layer[0] != 0) and 
                    (sorted_cur_layer_by_distance2cur_point[k][-1] > # max distance_Kth2cur
                        distance_euclidean([0, 0, 0], pointcloud[cur_idx][0: 3]) - distance_euclidean([0, 0, 0], cur_layer[0][0: 3]))
                ): # supplement inter layer
            idx_layer.insert(0, idx_layer[0]-1)
            cur_layer = np.r_[layers[idx_layer[0]], cur_layer]
            stats_offchip_access += np.size(layers[idx_layer[0]])
            ### Next step: Instance module 
            #############################################################
            # instance module of sorted according to distance2cur_point
            sorted_newlayer_by_distance2cur_point = sort_pointcloud_by_distance(pointcloud[cur_idx], layers[idx_layer[-1]])
           
            # 
            stats_computation_add += np.size(layers[idx_layer[-1]], 0)*3 + np.size(layers[idx_layer[-1]], 0)*2 + np.size(layers[idx_layer[-1]], 0)^2/2
            # sqrt = mul*4
            stats_computation_mul += np.size(layers[idx_layer[-1]], 0)*3 + np.size(layers[idx_layer[-1]], 0)*4
            stats_computation_cycle += np.size(layers[idx_layer[-1]], 0)^2/2 / ALU

            # uncertain (K-i) neighbor points need to seleted the miminum of (K-i) of cur_layer and (K-i) of newlayer.
            double_rest_points = np.r_[sorted_cur_layer_by_distance2cur_point[k: K], sorted_newlayer_by_distance2cur_point[0 : K-k]]

            # update farther positions of cur_layer_temp[i: K]
            double_rest_points = double_rest_points[np.lexsort(double_rest_points.T)]
            sorted_cur_layer_by_distance2cur_point[k: K] = double_rest_points[0: K-k]
            #############################################################

        while   (   (idx_layer[-1] != num_layer -1) and
                    (sorted_cur_layer_by_distance2cur_point[k][-1] > 
                        -(distance_euclidean([0, 0, 0], pointcloud[cur_idx][0: 3]) - distance_euclidean([0, 0, 0], cur_layer[-1][0: 3])))
                    ): # expand inter layer
            idx_layer.append(idx_layer[-1]+1)
            cur_layer = np.r_[cur_layer, layers[idx_layer[-1]]]
            stats_offchip_access += np.size(layers[idx_layer[-1]])

            #############################################################
            # instance module of sorted according to distance2cur_point
            sorted_newlayer_by_distance2cur_point = sort_pointcloud_by_distance(pointcloud[cur_idx], layers[idx_layer[-1]])
        
            # 
            stats_computation_add += np.size(layers[idx_layer[-1]], 0)*3 + np.size(layers[idx_layer[-1]], 0)*2 + + np.size(layers[idx_layer[-1]], 0)^2/2
            # sqrt = mul*4
            stats_computation_mul += np.size(layers[idx_layer[-1]], 0)*3 + np.size(layers[idx_layer[-1]], 0)*4
            stats_computation_cycle += np.size(layers[idx_layer[-1]], 0)^2/2 / ALU

            # pdb.set_trace()
            # uncertain (K-i) neighbor points need to seleted the miminum of (K-i) of cur_layer and (K-i) of newlayer.
            double_rest_points = np.r_[sorted_cur_layer_by_distance2cur_point[k: K], sorted_newlayer_by_distance2cur_point[0 : K-k]]

            # update farther positions of cur_layer_temp[i: K]
            double_rest_points = double_rest_points[np.lexsort(double_rest_points.T)]
            stats_computation_cycle += np.size(double_rest_points, 0)^2/2 / ALU

            sorted_cur_layer_by_distance2cur_point[k: K] = double_rest_points[0: K-k]
            #############################################################
    # print(sorted_cur_layer_by_distance2cur_point[0: 2])
    # distance_cur2loop_table_sort = () # update distance and index
    # print(sorted_cur_layer_by_distance2cur_point[0: K, 5])
    knn_map.append( np.r_[np.c_(pointcloud[cur_idx].reshape(1, 5), np.array([0]) ), sorted_cur_layer_by_distance2cur_point[0: K, 0: 6]] ) # store distance and index
    knn_map_idx.append( np.r_[np.array(cur_idx), sorted_cur_layer_by_distance2cur_point[0: K, 4]] )
    stats_offchip_access += len(knn_map_idx[-1]) # write knn_map_idx[-1]

    # cur_layer.shape[0]: distance of sorted_cur_layer_by_distance2cur_point and sorted_newlayer_by_distance2cur_point
    stats_onchip_storage = len(idx_layer) + np.size(cur_layer) + cur_layer.shape[0]

# print(knn_map[0: 5, 4: 6])



In [ ]:
# Check and Visualize

import matplotlib.pyplot as plt
# Creating figure
fig = plt.figure(figsize=(10, 7))
ax = plt.axes(projection="3d")

# Creating plot
ax.scatter3D(x, y, z, color="green")
plt.title("simple 3D scatter plot")

# show plot
plt.show()

